In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import random
import requests

load_dotenv()

True

## Generate Shot Descriptions

In [7]:
df = pd.read_csv("extracted_story_details.csv")

script = df.Script[0]
settings = df.Settings[0]
characters = df.Characters[0]

In [17]:
# Set up the OpenAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def generate_shot_descriptions(script, character_analysis, settings_analysis):
    """
    Generate a list of shot descriptions for a given script using GPT-4.
    Each shot description will include {setting: ...} and {character: ...} annotations.

    Args:
        script (str): The screenplay-style text for the scene.
        character_analysis (str): Freeform text describing characters and their roles.
        settings_analysis (str): Freeform text describing settings and context.

    Returns:
        list: A list of generated shot descriptions.
    """
    # System prompt to guide GPT-4
    system_prompt = """
    You are an expert in filmmaking and screenplay writing. Your task is to break down a scene script into detailed shot descriptions. Each shot must:

    1. Focus on a single moment or interaction.
    2. Use the exact {setting: ...} and {character: ...} annotations to maintain consistency.
    3. Avoid using any markdown formatting, including "plaintext" or code blocks.
    4. Avoid verbose prose and instead provide clear, actionable descriptions in the format below:
    
    Example shot descriptions:
    - {setting: village_square} {character: mr_summers} enters the square holding the black box. The camera pans to show his jovial expression as the villagers gather around.
    - {setting: village_square} {character: bobby_martin} dodges his {character: mother}'s grasp, running to a pile of stones with a mischievous laugh. The camera tracks his movement.

    Input Context:
    - The "Character Analysis" and "Settings Analysis" provide names and details for characters and settings.
    - The "Script" outlines the scene.

    Output Requirements:
    - Provide a list of shot descriptions, each starting with {setting: ...}.
    - Use {character: ...} tags for each character mentioned in the shot.
    - Use clear and concise descriptions without unnecessary detail.
    """

    # Prepare the user input
    user_input = f"""
    Script:
    {script}

    Character Analysis:
    {character_analysis}

    Settings Analysis:
    {settings_analysis}

    Please break the script into detailed shot descriptions.
    """

    # Call GPT-4 API
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ],
        model="gpt-4o",
    )

    # Access the response content correctly
    content = response.choices[0].message.content

    # Split the response into a list of shot descriptions
    shots = content.strip().split("\n")
    return [shot.strip() for shot in shots if shot.strip()]
# Example Input
script = """
Title: The Lottery

Scene: A village square, between the post office and the bank. It's the morning of June 27th. The day is clear and sunny, with the fresh warmth of a full-summer day. Flowers are blossoming, and the grass is richly green. Villagers are beginning to gather.

[The stage is set to resemble a small village square. A few men are already present, talking amongst themselves.]

[Enter WOMEN, greeting each other and their husbands, exchanging bits of gossip.]

WOMAN 1
[to her husband]
Did you hear about the Simmons' new cow? 

MAN 1
[chuckles]
Yes, and they say it's already broken the fence twice.

[More villagers join, and WOMEN begin to call their children.]

WOMAN 2
[calling]
Tommy! Come over here.

[CHILDREN, playing nearby, are reluctant to join their parents. They need to be called four or five times before they slowly make their way over.]

[BOBBY MARTIN ducks under his mother's grasping hand, runs back to a pile of stones, laughing. His FATHER steps forward.]

FATHER
[sharply]
Bobby! Get over here now.

[BOBBY MARTIN quickly returns, taking his place between his father and older brother.]

[MR. SUMMERS, a round-faced, jovial man, enters the square carrying a black wooden box. He is well-known for conducting village events.]

[The villagers' chatter quiets down as MR. SUMMERS takes his position.]

MR. SUMMERS
[smiling]
Morning, everyone! Let's begin, shall we? We wouldn't want to be late for noon dinner.

[The villagers murmur in agreement, forming a loose circle around MR. SUMMERS.]

[The scene captures a mix of anticipation and unease as they prepare for the lottery.]

[MR. SUMMERS places the black box on a stool, marking the start of the lottery.]

[The stage lights dim slightly to focus on the BLACK BOX, highlighting the significance of the event.]

[END OF SCENE]
"""

character_analysis = """
Name/Role: Mr. Summers
Characteristics: jovial, community-minded, diligent
Physical Description: round-faced
Description: Mr. Summers is the official who conducts the lottery. He runs the coal business and is actively involved in various civic activities. Despite his friendly demeanor, people feel sorry for him due to his unhappy personal life, as he has no children and his wife is known to be a scold.

Name/Role: Bobby Martin
Characteristics: playful, mischievous, obedient
Physical Description: none
Description: Bobby Martin is a young boy in the village. He is full of energy and playfulness, as shown by his initial reluctance to heed his mother’s calls while playing. He ultimately listens to his father and takes his place with his family, indicating his respect for authority and family.

Name/Role: Mr. Martin
Characteristics: authoritative, sharp, family-oriented
Physical Description: none
Description: Mr. Martin is Bobby's father. He is a figure of authority within his family, as seen when he sharply calls his son Bobby to order during the gathering. His presence indicates his role as a traditional family man who participates in the community event with his family.

Name/Role: Mrs. Martin
Characteristics: caring, persistent, motherly
Physical Description: none
Description: Mrs. Martin is Bobby's mother. She is depicted as a caring and persistent figure, trying multiple times to get Bobby’s attention and bring him back to the family. Her role conveys a sense of motherly duty and involvement in the family unit.

Name/Role: Oldest Brother of Bobby Martin
Characteristics: reserved, observant, family-oriented
Physical Description: none
Description: The oldest brother stands with his family during the lottery. His characterization is less detailed, but his presence suggests a more observant and quiet demeanor compared to his younger brother, Bobby. He follows his family’s lead in attending the lottery.
"""

settings_analysis = """
- **Time of Day and Weather**: A clear and sunny morning on June 27th, indicating the fresh warmth of a full-summer day with profusely blossoming flowers and richly green grass. - **Village Square**: Located between the post office and the bank, serving as the central gathering place for the villagers. - **Population Size**: A small village with about three hundred people, allowing the lottery event to be completed in less than two hours. - **Children's Gathering**: The children gather first, with a recently ended school term creating a sense of uneasy liberty. The boys engage in collecting stones and forming piles, while the girls stand and talk among themselves. Very small children are either rolling in the dust or holding onto older siblings.	
"""

# Generate shot descriptions
shot_descriptions = generate_shot_descriptions(script, character_analysis, settings_analysis)

# Print results
for shot in shot_descriptions:
    print(shot)


- {setting: village_square} A wide shot captures the serene and idyllic village square with clear skies and abundant summer foliage as villagers begin to gather.
- {setting: village_square} A medium shot of a few {character: men} already present, engaged in conversation and indicative of camaraderie.
- {setting: village_square} {character: women} enter the frame, seen greeting their husbands and exchanging bits of gossip, creating a lively atmosphere.
- {setting: village_square} {character: woman_1} speaks to {character: man_1} about the Simmons' cow. A close-up shows them chuckling together, capturing their casual rapport.
- {setting: village_square} The camera pans to capture more {character: villagers} joining the gathering, including {character: woman_2} calling for her child.
- {setting: village_square} {character: woman_2} repeatedly calls out for {character: tommy} who is reluctant to leave his play to join his family.
- {setting: village_square} {character: bobby_martin} is sho

In [16]:
"\n".join(shot_descriptions)

"```plaintext\n- {setting: village_square} {character: man_1} and other men converse near the village center. The camera captures their casual gestures and friendly demeanor in the morning sunlight.\n- {setting: village_square} A group of {character: woman}s enter the area, greeting their husbands with smiles and nods. The camera shows their friendly exchanges and animated gossip.\n- {setting: village_square} {character: woman_1} turns to {character: man_1}, sharing a lighthearted remark about their neighbors. The camera focuses on {character: man_1}'s amused reaction, highlighting the village camaraderie.\n- {setting: village_square} {character: woman_2} calls out to {character: children}, her voice rising above the chatter. The camera pans to the playing children, capturing their initial reluctance to join.\n- {setting: village_square} {character: bobby_martin} ducks playfully away from {character: mrs_martin}'s reach, running towards the pile of stones with a mischievous grin. The c

## Generate DALL-E Shot Prompts

In [24]:
def generate_dalle_prompts(shot_descriptions, character_analysis, settings_analysis):
    """
    Generate DALLE prompts for each shot description using GPT-4.
    Each DALLE prompt will include details from the shot description, character analysis, and settings analysis.

    Args:
        shot_descriptions (list): List of detailed shot descriptions.
        character_analysis (str): Freeform text describing characters and their roles.
        settings_analysis (str): Freeform text describing settings and context.

    Returns:
        list: A list of DALLE prompts corresponding to the shot descriptions.
    """
    # System prompt to guide GPT-4
    system_prompt = """
    You are an expert at crafting detailed visual prompts for DALL-E, specifically designed to create consistent and visually engaging storyboards. Your task is to take shot descriptions, along with character and setting analyses, and generate one high-quality DALL-E prompt for each shot.

Each DALL-E prompt must:
1. Clearly describe the scene, focusing on:
   - **Actions**: What is happening in the scene.
   - **Characters**: Include character names, appearances, and emotions when available. Ensure each character is distinct and consistent with the character analysis.
   - **Settings**: Describe the environment, lighting, and mood in detail.
2. Emphasize dynamic and engaging visuals:
   - Use active verbs and highlight emotions to create compelling imagery.
   - Ensure all visual elements contribute to the mood and story of the scene.
3. Maintain a consistent artistic style across all prompts:
   - Include attributes like "soft, natural lighting," "vibrant color palette," "painterly style," and "dynamic composition."
   - Avoid varying artistic attributes unless explicitly instructed.

### **Output Format**
- A standalone, natural-language description suitable for DALL-E. Do not include any annotations like {setting: ...} or {character: ...}.
- Avoid using markdown, code blocks, or any unnecessary symbols.

### **Prompt Structure**
"A digital painting of [scene description]. The scene features [character details] in [setting details], with [action details]. The style includes [lighting], [color palette], and [artistic attributes]."

### **Examples**
1. For a shot where villagers gather in the village square:
   - "A digital painting of a lively village square under the morning sun. The scene features villagers casually gathering, some chatting while others arrange stones in a pile, with vibrant green grass and blossoming flowers in the background. The style includes soft natural lighting, a warm color palette, and a painterly composition."

2. For a shot where Mr. Summers arrives with the black box:
   - "A digital painting of a jovial man entering a sunny village square, carrying a black wooden box. The scene captures his warm smile and confident posture as villagers turn to greet him. The style includes golden lighting, dynamic framing, and a focus on ceremonial tradition."

3. For a mischievous moment with Bobby Martin:
   - "A digital painting of a young boy darting away from his mother’s grasp, laughing mischievously as he runs toward a pile of stones. The scene is set in a sunny village square with vibrant green grass and the warm colors of summer. The style includes dynamic movement, bright lighting, and a focus on youthful energy."

---

### **Input Context**
You will receive:
- **Shot Descriptions**: Detailed descriptions of individual moments in a scene.
- **Character Analysis**: A list of characters with their names, appearances, and roles.
- **Settings Analysis**: A description of the environment, mood, and spatial elements.

### **Output Requirements**
For each shot description:
1. Generate a single DALL-E prompt following the format provided.
2. Ensure that the prompt captures the details and tone of the shot description while enhancing visual and emotional impact.
3. Do not include any unnecessary formatting or redundant information.

    """

    # Prepare the user input
    user_input = f"""
    Shot Descriptions:
    {shot_descriptions}

    Character Analysis:
    {character_analysis}

    Settings Analysis:
    {settings_analysis}

    Please generate one DALLE prompt for each shot description.
    """

    # Call GPT-4 API
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ],
        model="gpt-4o",
    )

    # Access the response content correctly
    content = response.choices[0].message.content

    # Split the response into individual DALLE prompts
    dalle_prompts = [prompt.strip() for prompt in content.split("\n") if prompt.strip()]
    return dalle_prompts

In [25]:
dalle_prompts = generate_dalle_prompts(shot_descriptions, character_analysis, settings_analysis)

In [26]:
dalle_prompts

['A digital painting of a serene and idyllic village square under a clear blue sunny sky. The scene features villagers slowly gathering amid abundant summer foliage, with the warm morning light infusing a sense of tranquility. The style includes soft natural lighting, a rich green and floral color palette, and a painterly composition capturing a peaceful community moment.',
 'A digital painting of a small group of men engaged in lively conversation in the village square. The scene captures their camaraderie as they discuss with animated expressions under the warm summer sun. The style includes soft, natural lighting, vibrant colors, and a dynamic composition highlighting the friendly atmosphere.',
 'A digital painting of women entering a bustling village square, greeting their husbands and exchanging gossip. The lively atmosphere is enhanced by their animated gestures and cheerful expressions, set against the backdrop of the vibrant summer morning. The style includes soft, natural ligh

In [ ]:
def generate_and_save_images(prompts, output_directory, panel_sizes=None):
    """
    Generate and save images using OpenAI's DALL-E 3 API based on a list of prompts.

    Args:
        prompts (list): A list of text prompts to generate images for.
        output_directory (str): The directory to save the generated images.
        panel_sizes (list): A list of sizes for the images ("256x256", "512x512", "1024x1024").
                            If None, defaults to "512x512" for all prompts.

    Returns:
        None: Saves images to the specified directory.
    """
    # Default panel sizes to 512x512 if none are provided
    if panel_sizes is None:
        panel_sizes = ["512x512"] * len(prompts)
    elif len(panel_sizes) != len(prompts):
        raise ValueError("The number of panel sizes must match the number of prompts.")

    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Iterate through prompts and panel sizes
    for i, (prompt, size) in enumerate(zip(prompts, panel_sizes)):
        try:
            # Generate image using the DALL-E 3 API
            response = client.images.generate(
                model="dall-e-3",
                prompt=prompt,
                n=1,
                size=size
            )

            image_url = response.data[0].url

            # Download the image content
            image_filename = os.path.join(output_directory, f"panel_{i+1}_{size}.png")
            image_response = requests.get(image_url)
            image_response.raise_for_status()

            # Save the image to the specified directory
            with open(image_filename, "wb") as image_file:
                image_file.write(image_response.content)

            print(f"Image saved: {image_filename}")

        except AttributeError as attr_err:
            print(f"Attribute error for prompt {i+1}: {attr_err}")
        except Exception as e:
            print(f"Error generating or saving image for prompt {i+1}: {e}")


In [ ]:
# Define possible panel sizes
# 1024x1024, 1024x1792 or 1792x1024
panel_options = ['1024x1024', '1024x1792', '1792x1024']

# List to store the chosen panel sizes
panel_sizes = []

for _ in dalle_prompts:
    # Randomly choose a square, landscape, or portrait panel
    panel_type = random.choice(panel_options)
    panel_sizes.append(panel_type)

# Generate and save images with the chosen panel sizes
output_directory = "./storyboard_images"
generate_and_save_images(dalle_prompts, output_directory, panel_sizes)

Error generating or saving image for prompt 1: name 'image_url' is not defined
Error generating or saving image for prompt 2: name 'image_url' is not defined
Error generating or saving image for prompt 3: name 'image_url' is not defined


In [31]:
len(dalle_prompts)

15